<a href="https://colab.research.google.com/github/daniivelascoo/ifp-programacion-ia/blob/main/Teoria_1_3_D_Merge_Student.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧠 Teoría Bloque D: Cruzando Bases de Datos (Merge)
**Sprint 1.3 - Pandas Deep Dive**

En el mundo real, la información está fragmentada.
*   Una tabla tiene los **Clientes**.
*   Otra tabla tiene los **Pedidos**.
*   Otra tabla tiene los **Productos**.

Para analizar todo junto, necesitamos unir estas tablas. Esto se llama **JOIN** en SQL, y **MERGE** en Pandas.

### 🎯 Objetivos:
1.  **Relaciones:** Entender las claves comunes (ID).
2.  **Inner Join:** La intersección (Solo lo que coincide).
3.  **Left Join:** La prioridad (No perder datos de la tabla principal).

## 1. Preparación del Escenario

Vamos a crear dos DataFrames que simulan una base de datos de comercio.
*   **Clientes:** Tenemos 3 clientes registrados (Ana, Bob, Charlie).
*   **Pedidos:** Tenemos 4 pedidos. Fíjate bien:
    *   Hay un pedido del cliente 99 (Un cliente que NO existe en nuestra base de datos).
    *   Charlie (id 3) NO ha hecho ningún pedido.

In [1]:
import pandas as pd

# Tabla A: CLIENTES
clientes = pd.DataFrame({
    'id_cliente': [1, 2, 3],
    'nombre': ['Ana', 'Bob', 'Charlie'],
    'email': ['ana@test.com', 'bob@test.com', 'charlie@test.com']
})

# Tabla B: PEDIDOS
pedidos = pd.DataFrame({
    'id_pedido': [101, 102, 103, 104],
    'id_cliente': [1, 1, 2, 99], # El 99 es un misterio
    'importe': [50, 20, 100, 200]
})

print("--- Tabla Clientes ---")
display(clientes)
print("\n--- Tabla Pedidos ---")
display(pedidos)

--- Tabla Clientes ---


,id_cliente,nombre,email
0,1,Ana,ana@test.com
1,2,Bob,bob@test.com
2,3,Charlie,charlie@test.com



--- Tabla Pedidos ---


,id_pedido,id_cliente,importe
0,101,1,50
1,102,1,20
2,103,2,100
3,104,99,200


## 2. Inner Join (La Intersección Estricta)

Queremos una tabla limpia: **Solo pedidos de clientes que conocemos.**
*   Si un cliente no ha comprado (Charlie), no sale.
*   Si un pedido es de un desconocido (99), no sale.

Usamos `how='inner'`.

In [2]:
# Sintaxis: pd.merge(IZQUIERDA, DERECHA, on='CLAVE_COMUN', how='TIPO')

# Unimos Pedidos (Izquierda) con Clientes (Derecha)
# La clave común es 'id_cliente'

merge_inner = pd.merge(
    pedidos,
    clientes,
    on='id_cliente',   # TODO: Pon el nombre de la columna común
    how='inner'
)

print("--- INNER JOIN (Solo coincidencias) ---")
display(merge_inner)

# Observa: Han desaparecido el pedido 104 (cliente 99) y el cliente Charlie (3).

--- INNER JOIN (Solo coincidencias) ---


,id_pedido,id_cliente,importe,nombre,email
0,101,1,50,Ana,ana@test.com
1,102,1,20,Ana,ana@test.com
2,103,2,100,Bob,bob@test.com


## 3. Left Join (El Estándar en Data Science)

Imagina que eres el jefe de ventas. Quieres un informe de **TODOS** los pedidos, incluso si hay un error y no sabemos el nombre del cliente. No podemos permitir que el pedido desaparezca del reporte financiero.

Usamos `how='left'` (Mantener todo lo de la tabla izquierda).

In [3]:
# Unimos Pedidos (Izquierda) con Clientes (Derecha)
# Queremos priorizar la tabla de PEDIDOS.

merge_left = pd.merge(
    pedidos,
    clientes,
    on='id_cliente',
    how='left' # TODO: Escribe 'left'
)

print("--- LEFT JOIN (Todos los pedidos) ---")
display(merge_left)

# Observa la última fila (Pedido 104):
# El nombre y email salen como NaN (No a Number / Vacío) porque el cliente 99 no existe.
# PERO el pedido de 200€ sigue ahí. ¡No hemos perdido dinero en el reporte!

--- LEFT JOIN (Todos los pedidos) ---


,id_pedido,id_cliente,importe,nombre,email
0,101,1,50,Ana,ana@test.com
1,102,1,20,Ana,ana@test.com
2,103,2,100,Bob,bob@test.com
3,104,99,200,NaN,NaN


## 4. Reflexión Visual

Si hubiéramos hecho lo contrario (Clientes a la izquierda y Pedidos a la derecha con Left Join), ¿qué habría pasado?

Que saldría **Charlie** (aunque no tenga pedidos, saldría con importe `NaN`) y desaparecería el pedido del cliente 99.

**Regla de Oro:** Pon siempre a la izquierda la tabla "Maestra" (la que contiene los hechos que quieres analizar).